第一遍运行时，暂不设置 weightScale

In [ ]:
import matplotlib.pyplot as plt

In [2]:
from mxnet import nd, gluon, autograd

In [3]:
def transform(x, y):
    return x.astype('float32')/255, y.astype('float32')

In [4]:
mnistTrain = gluon.data.vision.FashionMNIST(root='../workdir/mnist/', train=True, transform=transform)
mnistTest = gluon.data.vision.FashionMNIST(root='../workdir/mnist/', train=False, transform=transform)

In [ ]:
def showImages(xs):
    n = xs.shape[0]
    _, ax = plt.subplots(1, n, figsize=(15, 15))
    for i in range(n):
        ax[i].imshow(xs[i].reshape((28, 28)).asnumpy())
        ax[i].axes.get_xaxis().set_visible(False)
        ax[i].axes.get_yaxis().set_visible(False)
    plt.show()

In [ ]:
def getLabels(ys):
    labels = [
        't-shirt', 'trouser', 'pullover', 'dress,', 'coat',
        'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot'
    ]
    result = []
    for i in ys:
        result.append(labels[int(i)])
    return result

In [5]:
batchSize = 256
trainData = gluon.data.DataLoader(mnistTrain, batchSize, shuffle=True)
testData = gluon.data.DataLoader(mnistTest, batchSize, shuffle=False)

拍脑袋拍出来的

In [ ]:
numInputs = 28*28
numOutputs = 10
numHidden = 256 #美其名曰 根据经验

In [ ]:
w1 = nd.random_normal(shape=(numInputs, numHidden))
b1 = nd.zeros(numHidden)
w2 = nd.random_normal(shape = (numHidden, numOutputs))
b2 = nd.zeros(numOutputs)
params = [w1, b1, w2, b2]

In [ ]:
for param in params:
    param.attach_grad()

In [ ]:
def relu(x):
    return nd.maximum(x, 0)

In [ ]:
def net(x):
    x = x.reshape((-1, numInputs))
    h1 = relu(nd.dot(x, w1) + b1)
    output = nd.dot(h1, w2) + b2
    return output

In [ ]:
def SGD(params, eta):
    for param in params:
        param[:] = param - eta * param.grad

In [ ]:
def accuracy(yhat, y):
    return nd.mean(yhat.argmax(axis=1)==y).asscalar()

In [ ]:
def evaluateAcc(dataIterator, net):
    acc = 0.
    for x, y in dataIterator:
        yhat = net(x)
        acc += accuracy(yhat, y)
    return acc / len(dataIterator)

In [ ]:
evaluateAcc(testData, net)

In [ ]:
softmaxCrossEntropy = gluon.loss.SoftmaxCrossEntropyLoss()

In [ ]:
eta = 0.5
epochs = 5

In [ ]:
for epoch in range(epochs):
    trainLoss = 0.
    trainAcc = 0.
    for x, y in trainData:
        with autograd.record():
            yhat = net(x)
            loss = softmaxCrossEntropy(yhat, y)
        loss.backward()
        SGD(params, eta/batchSize)
        trainLoss += nd.mean(loss).asscalar()
        trainAcc += accuracy(yhat, y)
        
    testAcc = evaluateAcc(testData, net)
    print('Epoch %d. Loss: %f, Train acc: %f, Test acc %f' % (
        epoch, trainLoss/len(trainData), trainAcc/len(trainData), testAcc))

In [ ]:
xs, ys = mnistTest[0:9]
showImages(xs)
print('Ground truth: ')
print(getLabels(ys))

yhat = net(xs).argmax(axis=1)
print('Prediction: ')
print(getLabels(yhat.asnumpy()))